In [ ]:
import random
import math
import numbers
import platform
import copy

import numpy as np
import PIL
from PIL import Image, ImageOps
import matplotlib.pyplot as plt
plt.rcParams["savefig.bbox"] = 'tight'


import cv2


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.autograd as autograd
from torch.utils.data import Dataset, DataLoader
from torch import Tensor
from torch.jit import script, trace

import torchvision.transforms as transforms
import torchvision.transforms.functional as tF

#device = torch.device('cuda')
device = torch.device('cpu')

# img = cv2.imread('data/0/frame_1.jpg', cv2.IMREAD_COLOR)
# img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
# plt.axis('off')
# plt.imshow(img)

# def draw_rect_perm(image, arr):
#     top_left, bottom_right = (arr[0], arr[1]), (arr[0]+arr[2], arr[1] + arr[3])
#     cv2.rectangle(image, top_left, bottom_right, (255,0,0), 2)

# roi = [[502, 344, 61, 86],
#       [627, 427, 193, 93],
#       [522, 654, 90, 85],
#       [704, 594, 90, 80],
#       [545, 577, 81, 75],
#       [460, 700, 52, 80]]

# def show_roi(image):
#     img2 = image.copy()
#     for i in range(6):
#         draw_rect_perm(img2, roi[i])
#     draw_rect_perm(img2, [400, 300, 500, 500])
#     plt.axis('off')
#     plt.imshow(img2)

# show_roi(img)
# plt.show()
# exit()

import torchvision
from torchvision.models.segmentation.segmentation import fcn_resnet101
from torchvision.io.image import read_image
from torchvision.models.segmentation import fcn_resnet50, deeplabv3_resnet50
from torchvision.transforms.functional import to_pil_image
from torchvision.utils import save_image

def crop500(image):
    return tF.crop(image, 300, 400, 500, 500)

# i = 0
# def save(imgs):
#     global i
#     if not isinstance(imgs, list):
#         imgs = [imgs]
#     for _, img in enumerate(imgs):
#         img = img.detach()
#         save_image(img, f't{i}.jpg')
#         i += 1

# resize = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Lambda(crop500),
#     #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
#     ])
# dataset = torchvision.datasets.ImageFolder('shiti/', transform=resize)
# dataloader = DataLoader(dataset, batch_size=5)
# for data, _ in dataloader:
#     save([data[i] for i in range(data.shape[0])])
# # exit()

# def cropCUS(image):
#     return tF.crop(image, 344, 502, 61, 86)


#####################################################################
#img = read_image('/content/data/frame_1.jpg')
# resize = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Lambda(crop500),
#     #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
#     ])
# dataset = torchvision.datasets.ImageFolder('data/', transform=resize)
# dataloader = DataLoader(dataset, batch_size=2)


# model = fcn_resnet50(pretrained=True, progress=False)
# model = model.eval()
# model = model.to(device)

# for data, _ in dataloader:
#     data = data.to(device)
#     out = model(data)['out']
#     print(model(data)['aux'].shape)
#     print(model(data)['out'].shape)
#     break

# #normalized_batch = F.normalize()
# sem_classes = [
#     '__background__', 'aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus',
#     'car', 'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike',
#     'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor'
# ]
# sem_class_to_idx = {cls: idx for (idx, cls) in enumerate(sem_classes)}
# normalized_masks = torch.nn.functional.softmax(out, dim=1)

# human_mask = [normalized_masks[img_idx, sem_class_to_idx['person']]
#     for img_idx in range(data.shape[0])]



/usr/local/lib/python3.7/dist-packages/torchvision/models/segmentation/segmentation.py:8: UserWarning: The 'torchvision.models.segmentation.segmentation' module is deprecated since 0.12 and will be removed in 0.14. Please use the 'torchvision.models.segmentation' directly instead.
  "The 'torchvision.models.segmentation.segmentation' module is deprecated since 0.12 and will be removed in "


In [ ]:
w = h = 500
batch_size = 8
train_dir = "/content/shitout"
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

transformAnn = transforms.ToTensor()


batches = []
for i in range(batch_size):
    img = cv2.imread(f"/content/shitout/t{i}.jpg")
    lbl = cv2.imread(f"/content/verify/t{i}.jpg")

    ann_map = np.zeros(img.shape[0:2], bool)
    for i in range(500):
        for j in range(500):
          ann_map[i,j] = np.any(lbl[i,j] < 250)
    #plt.imshow(ann_map)
    img = transform(img)
    ann_map = transformAnn(ann_map)
    batches.append((img,ann_map))

def load_batch():
    imgs = torch.zeros([batch_size,3,h,w])
    ann = torch.zeros([batch_size,h,w])

    for i in range(batch_size):
      imgs[i], ann[i] = batches[i][0], batches[i][1]

    return imgs, ann


In [ ]:

model = deeplabv3_resnet50(pretrained=True)
model.classifier[4] = nn.Conv2d(256, 2, kernel_size=(1,1), stride=(1,1))
model = model.to(device)

In [ ]:
########### retrain model
lr = 1e-4


# first train the last layer
optimizer = torch.optim.Adam(model.classifier[4].parameters(),
                             lr=lr)

model.train()
for iter in range(500):
    imgs,ann = load_batch()
    optimizer.zero_grad()
    with torch.set_grad_enabled(True):
        imgs = torch.autograd.Variable(imgs,requires_grad=False).to(device)
        ann = torch.autograd.Variable(ann, requires_grad=False).to(device)
        pred = model(imgs)['out']

        criterion = torch.nn.CrossEntropyLoss()
        loss=criterion(pred,ann.long())
        loss.backward()
        optimizer.step()

    if iter % 10 == 0:
        print(loss.item())


    #running_loss.append(loss.item())      

torch.save(model.state_dict(), "naive0.torch")

# then fine tune the entire model


0.5312653183937073
0.47667768597602844
0.42740321159362793
0.38468295335769653
0.34687575697898865
0.3145785331726074
0.286141037940979
0.26163703203201294
0.24040980637073517
0.2216147929430008
0.20556199550628662
0.19153280556201935
0.1788620948791504
0.16748961806297302
0.15755106508731842
0.14868180453777313
0.14084526896476746
0.13375510275363922
0.12776906788349152
0.12151747196912766
0.11636745929718018
0.11171677708625793
0.10690564662218094
0.10316788405179977
0.09963300079107285
0.09633109718561172
0.09339546412229538
0.09027513861656189
0.08748038858175278
0.08522737771272659
0.08341772109270096
0.0810476541519165
0.0791003480553627
0.07691626250743866
0.07552483677864075
0.07418457418680191
0.07277900725603104
0.07093112170696259
0.06987183541059494
0.06870714575052261
0.06758219003677368
0.06613949686288834
0.06563197076320648
0.06436093896627426
0.06338363885879517
0.06238926947116852
0.061885423958301544
0.06075203791260719
0.060325298458337784
0.059818923473358154


In [ ]:
model = deeplabv3_resnet50(pretrained=True)
model.classifier[4] = nn.Conv2d(256, 2, kernel_size=(1,1), stride=(1,1))
model.load_state_dict(torch.load("naive1.torch",map_location=torch.device('cpu')))
model = model.to(device)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=DeepLabV3_ResNet50_Weights.COCO_WITH_VOC_LABELS_V1`. You can also use `weights=DeepLabV3_ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
import gc
gc.collect()

lr = 1e-5


# first train the last layer
optimizer = torch.optim.Adam(model.parameters(),
                             lr=lr)

model.train()
for iter in range(150):
    imgs,ann = load_batch()
    optimizer.zero_grad()
    with torch.set_grad_enabled(True):
        imgs = torch.autograd.Variable(imgs,requires_grad=False).to(device)
        ann = torch.autograd.Variable(ann, requires_grad=False).to(device)
        pred = model(imgs)['out']

        criterion = torch.nn.CrossEntropyLoss()
        loss=criterion(pred,ann.long())
        loss.backward()
        optimizer.step()

    if iter % 10 == 0:
        print(loss.item())


    #running_loss.append(loss.item())      

torch.save(model.state_dict(), "naive1.torch")

# then fine tune the entire model

0.06729988753795624
0.048897214233875275
0.04241541028022766
0.03817346319556236
0.0354403592646122
0.033113740384578705
0.03131159022450447
0.029709307476878166
0.02817200869321823
0.026802441105246544
0.025583485141396523
0.024541478604078293
0.023612130433321
0.022763174027204514
0.021989120170474052


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def show(imgs):
    if not isinstance(imgs, list):
        imgs = [imgs]
    fix, axs = plt.subplots(ncols=len(imgs), squeeze=False)
    for i, img in enumerate(imgs):
        img = img.detach()
        img = tF.to_pil_image(img)
        axs[0, i].imshow(np.asarray(img), aspect='auto')
        axs[0, i].set(xticklabels=[], yticklabels=[], xticks=[], yticks=[])


In [ ]:
resize = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(crop500),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ])
dataset = torchvision.datasets.ImageFolder('data/', transform=resize)
dataloader = DataLoader(dataset, batch_size=5)

for data, _ in dataloader:
    data = data.to(device)
    out = model(data)['out']
    break

normalized_masks = torch.nn.functional.softmax(out, dim=1)
human_mask = [normalized_masks[i, 1] for i in range(data.shape[0])]


# #print(normalized_masks[0])


resize = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(crop500),
    #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ])
dataset = torchvision.datasets.ImageFolder('data/', transform=resize)
dataloader = DataLoader(dataset, batch_size=5)

for data, _ in dataloader:
    data = data.to(device)
    break
show([data[img_idx] for img_idx in range(data.shape[0])])
show(human_mask)

In [ ]:
threshold = 20

roi = [[502, 344, 61, 86],
      [627, 427, 193, 93],
      [522, 654, 90, 85],
      [704, 594, 90, 80],
      [545, 577, 81, 75],
      [460, 700, 52, 80]]

#[400, 300, 500, 500]

region = [np.zeros((500,500), dtype=bool) for _ in range(6)]
for i in range(6):
    a = roi[i][0]-400
    b = roi[i][1]-300
    region[i][b:b+roi[i][3], a:a+roi[i][2]] = 1

# for i in range(1, 13502):
#     img = f"/content/drive/MyDrive/musashi_challenge/frame_{i}.jpg"

batches = []
model.eval()
# for i in range(1, 13502):
#     img = cv2.imread(f"/content/drive/MyDrive/musashi_challenge/frame_{i}.jpg")
#     #plt.imshow(ann_map)
#     img = resize(img)
#     print(img.shape)
#     tmp = torch.zeros([1,3,h,w])
#     tmp[0] = img
#     img = tmp
#     out = model(img)['out']

#     normalized_masks = torch.nn.functional.softmax(out, dim=1)
#     human_mask = normalized_masks[0, 1]
#     print(normalized_masks.shape)
#     print(human_mask.shape)
#     show(human_mask)
#     break;

for i in range(1, 13502):
    img = cv2.imread(f"/content/drive/MyDrive/musashi_challenge/frame_{i}.jpg")
    #plt.imshow(ann_map)
    img = resize(img)
    print(img.shape)
    tmp = torch.zeros([1,3,h,w])
    tmp[0] = img
    img = tmp
    out = model(img)['out']

    normalized_masks = torch.nn.functional.softmax(out, dim=1)
    human_mask = normalized_masks[0, 1]
    print(normalized_masks.shape)
    print(human_mask.shape)
    show(human_mask)
    break;

TypeError: ignored